In [90]:
import numpy as np
import glob
import os
import json
from ndview import ndv
import matplotlib.pyplot as plt
#from utils import load_mat

import tifffile as tf
import zarr

%matplotlib widget
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Directory with tif files, recorded with ScanImage

In [91]:
wdir = r"D:\Userdata\johannes\tmp"
tflist = glob.glob(os.path.join(wdir,'*tif')) 
tflist

['D:\\Userdata\\johannes\\tmp\\pv-0b-211012_hydrophoneinput.tif']

In [92]:
fn = tflist[-1]
fn

'D:\\Userdata\\johannes\\tmp\\pv-0b-211012_hydrophoneinput.tif'

### Load as Zarr array via tiffile module

In [93]:
scan = tf.TiffFile(fn)
meta = scan.scanimage_metadata
store = scan.aszarr()
data_all = zarr.open(store, mode='a',chunks=True) # can specify chunking that is optimal for planned operations
data_all

<zarr.core.Array (42098, 3, 512, 512) int16>

In [94]:
data_all.chunks

(1, 1, 512, 512)

Metadata

In [95]:
mt = {}
mt['nvls'] = meta['FrameData']['SI.hFastZ.numVolumes']
mt['nslcs'] = meta['FrameData']['SI.hStackManager.numSlices']
if isinstance(meta['FrameData']['SI.hChannels.channelSave'],list):
    mt['chnames'] = [c[0] for c in meta['FrameData']['SI.hChannels.channelSave']] 
    mt['nch'] = len(mt['chnames'])
else:
    mt['chnames'] = meta['FrameData']['SI.hChannels.channelSave']
    mt['nch'] = 1
mt['npslc'] = meta['FrameData']['SI.hStackManager.framesPerSlice']
mt['h'] = meta['FrameData']['SI.hRoiManager.linesPerFrame']
mt['w'] =  meta['FrameData']['SI.hRoiManager.pixelsPerLine']
if meta['FrameData']['SI.hFastZ.enable']:
    mt['nslcs'] = meta['FrameData']['SI.hFastZ.numFramesPerVolume']
mt

{'nvls': 1358,
 'nslcs': 31,
 'chnames': [1, 2, 3],
 'nch': 3,
 'npslc': 1,
 'h': 512,
 'w': 512}

Load only a subset of the data (to avoid buffer overflow)

In [99]:
# number of volumes to load
nvls_load = 20
data = data_all[:nvls_load*mt['nslcs']]
data.shape

(620, 3, 512, 512)

In [100]:
# reshape
order = (nvls_load,mt['nslcs'],mt['npslc'],mt['nch'],mt['h'],mt['w'])
data = data.reshape(order)

In [101]:
# view
ndv(data)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [102]:
# mean projection
ar = np.mean(data[:,:,0,2,:,:], axis = (2,3))

In [103]:
plt.figure(), plt.plot(ar.T)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<Figure size 640x480 with 1 Axes>,
  <matplotlib.lines.Line2D at 0x1b4cf1c96d0>])

In [104]:
meta

{'version': 3,
 'FrameData': {'SI.LINE_FORMAT_VERSION': 1,
  'SI.TIFF_FORMAT_VERSION': 3,
  'SI.VERSION_COMMIT': 'ca9e5a08f36348e2a0551efd86341b8fec5367fd',
  'SI.VERSION_MAJOR': '5.6',
  'SI.VERSION_MINOR': '1',
  'SI.acqState': 'grab',
  'SI.acqsPerLoop': 1,
  'SI.extTrigEnable': False,
  'SI.hBeams.beamCalibratedStatus': False,
  'SI.hBeams.directMode': False,
  'SI.hBeams.enablePowerBox': False,
  'SI.hBeams.flybackBlanking': True,
  'SI.hBeams.interlaceDecimation': 1,
  'SI.hBeams.interlaceOffset': 0,
  'SI.hBeams.lengthConstants': inf,
  'SI.hBeams.powerBoxEndFrame': inf,
  'SI.hBeams.powerBoxStartFrame': 1,
  'SI.hBeams.powerBoxes.rect': [0.25, 0.25, 0.5, 0.5],
  'SI.hBeams.powerBoxes.powers': nan,
  'SI.hBeams.powerBoxes.name': '',
  'SI.hBeams.powerBoxes.oddLines': True,
  'SI.hBeams.powerBoxes.evenLines': True,
  'SI.hBeams.powerLimits': 100,
  'SI.hBeams.powers': 10,
  'SI.hBeams.pzAdjust': False,
  'SI.hBeams.pzCustom': [],
  'SI.hBeams.stackEndPower': nan,
  'SI.hBeams.sta

In [ ]:
#slow
plt.figure()
hist, bins = np.histogram(data.flatten(), bins = 100)
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:]) / 2
hist, bins = np.histogram(data[:,:,:,0].flatten(), bins = bins)
plt.bar(center, hist, align='center', width=width)
hist, bins = np.histogram(data[:,:,:,1].flatten(), bins = bins)
plt.bar(center, hist, align='center', width=width)
hist, bins = np.histogram(data[:,:,:,2].flatten(), bins = bins)
plt.bar(center, hist, align='center', width=width)

In [ ]:
d, h = os.path.split(wdir)
outdir = os.path.join(d,h+'_processingNeural')
try:
    os.makedirs(outdir)
except OSError:
    print ("Creation of the directory %s failed" % outdir)
else:
    print ("Successfully created the directory %s" % outdir)

In [139]:
scan

In [ ]:
from ScanImageTiffReader import ScanImageTiffReader 
scan = ScanImageTiffReader(fn)
data = scan.data()
meta = scan.metadata()